In [9]:
import torch
import torchaudio
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np
import fairseq 
from pathlib import Path

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

In [10]:
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([str(Path.home())+'/Daniyal/work/model/kannada_pretrained_1400h.pt'])
model = model[0]
model.eval()

/home/ubuntu/miniconda3/envs/profanity_detection/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Wav2Vec2Model(
  (feature_extractor): ConvFeatureExtractionModel(
    (conv_layers): ModuleList(
      (0): Sequential(
        (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
        (1): Dropout(p=0.0, inplace=False)
        (2): Sequential(
          (0): TransposeLast()
          (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (2): TransposeLast()
        )
        (3): GELU(approximate='none')
      )
      (1-4): 4 x Sequential(
        (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
        (1): Dropout(p=0.0, inplace=False)
        (2): Sequential(
          (0): TransposeLast()
          (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (2): TransposeLast()
        )
        (3): GELU(approximate='none')
      )
      (5-6): 2 x Sequential(
        (0): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
        (1): Dropout(p=0.0, inplace=False)
        (2): Sequential(
          (0): TransposeLast()
          (1): Fp32Laye

In [11]:
class AudioDataset (Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd. read_csv (csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self) :
        return len(self.annotations)

    def __getitem__(self, index) :

        # 16000 * 30
        file_path = os.path.join(self. root_dir, self.annotations. iloc[index, 0])
        waveform, sr = torchaudio.load(file_path)


        #waveform= truncate_or_pad_waveform(waveform, target_length=100000)
        
        if self .annotations. iloc [index, 1]=='Yes':
            y_label = torch. tensor(1)
        else:
            y_label = torch. tensor (0)
        
        if self.transform:
            waveform = self.transform(waveform)
        
        
        waveform=model.feature_extractor(waveform)
        #max_pool, _ = torch.max(waveform, dim=2)

        return (waveform,self.annotations. iloc[index, 0], y_label)

In [12]:
test_dataset = AudioDataset (csv_file = '/home/ubuntu/Daniyal/work/data/Kannada_test.csv', root_dir = "/home/ubuntu/Daniyal/work/data/Prima/SC_audio_Kannada")

In [13]:
test_loader = DataLoader (dataset=test_dataset, batch_size=1, shuffle=True)

In [16]:
#for test dataset

labels=[]
names=[]
for batch_idx, (data, name, label) in enumerate(test_loader):


    fromatted_data= data.cpu().detach().numpy().reshape(512, -1)
    
    # # For demonstration, let's stop after 5 batches
    # if batch_idx >= 4:
    #     break

    labels.append(label.cpu().detach().numpy()[0])
    names.append(name[0].split('.')[0]+'.npy')
    print(label, name)

    np.save('/home/ubuntu/Daniyal/work/data/features/test/'+name[0].split('.')[0]+'.npy', fromatted_data)

np_csv = {
    "Name": names,
    "Label": labels
}
np_df = pd.DataFrame(np_csv)

np_df.to_csv('/home/ubuntu/Daniyal/work/data/features/test/np_test.csv', index=False)

tensor([1]) ('Abuse_683.wav',)
tensor([1]) ('Abuse_553.wav',)
tensor([0]) ('Abuse_610.wav',)
tensor([0]) ('Abuse_145.wav',)
tensor([0]) ('Abuse_900.wav',)
tensor([1]) ('Abuse_558.wav',)
tensor([1]) ('Abuse_1061.wav',)
tensor([1]) ('Abuse_677.wav',)
tensor([1]) ('Abuse_1074.wav',)
tensor([1]) ('Abuse_1196.wav',)
tensor([0]) ('Abuse_41.wav',)
tensor([1]) ('Abuse_912.wav',)
tensor([1]) ('Abuse_873.wav',)
tensor([0]) ('Abuse_859.wav',)
tensor([0]) ('Abuse_1190.wav',)
tensor([1]) ('Abuse_751.wav',)
tensor([1]) ('Abuse_459.wav',)
tensor([0]) ('Abuse_686.wav',)
tensor([0]) ('Abuse_365.wav',)
tensor([0]) ('Abuse_552.wav',)
tensor([0]) ('Abuse_846.wav',)
tensor([0]) ('Abuse_964.wav',)
tensor([0]) ('Abuse_524.wav',)
tensor([0]) ('Abuse_246.wav',)
tensor([1]) ('Abuse_273.wav',)
tensor([0]) ('Abuse_589.wav',)
tensor([0]) ('Abuse_123.wav',)
tensor([0]) ('Abuse_916.wav',)
tensor([0]) ('Abuse_599.wav',)
tensor([0]) ('Abuse_688.wav',)
tensor([0]) ('Abuse_7.wav',)
tensor([0]) ('Abuse_225.wav',)
tensor(

In [18]:
temp=pd.read_csv('/home/ubuntu/Daniyal/work/data/features/test/np_test.csv')
temp

,Name,Label
0,Abuse_683.npy,1
1,Abuse_553.npy,1
2,Abuse_610.npy,0
3,Abuse_145.npy,0
4,Abuse_900.npy,0
...,...,...
364,Abuse_905.npy,0
365,Abuse_153.npy,0
366,Abuse_268.npy,0
367,Abuse_1084.npy,0
